# Evaluation & Promotion Of Models

This notebook loads the champion and challenger models and run inference on evaluation dataset. Based of the perfomance criteria it decides whether to promote the challenger to champion or not.

In [ ]:
# Import python packages
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from snowflake.snowpark.context import get_active_session
from snowflake.ml.registry import Registry
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
# Set up the database, schema and model registry
database_name='DEV_AUTOMATION_DEMO'
schema_name='CHAMPION_CHALLENGER'
session.use_database(database_name)
session.use_schema(schema_name)  

# Initialize registry
registry = Registry(session=session, 
                           database_name=database_name, 
                           schema_name=schema_name)  
print(f"✅ Environment ready: {database_name}.{schema_name}")

# Evaluation Data
Get the holdout dataset from week 16-19 saved in the table named evaluation_data.

In [ ]:
eval_dataset = session.table('evaluation_data').to_pandas()

# 📋 Prepare Features and Target
feature_cols = [col for col in eval_dataset.columns 
               if col not in {'application_date', 'week_number', 'approved'}]

#prepare dataset to evaluate performance
X_eval = eval_dataset[feature_cols]
y_true = eval_dataset['approved']

# Model Details
Get the champion and challenger versions of model from the Snowflake model registry.

In [ ]:
#Get the model
model = registry.get_model("CREDIT_APPROVAL")

# 🏆 Get Live (Champion) Model Version
live_version = model.get_tag("live_version")
print(f"🏆 Current live model version: {live_version}")

# 🥊 Get Challenger Model Version (if exists)
try:
    challenger_version = model.get_tag("challenger_version")
    print(f"🥊 Current challenger version: {challenger_version}")
except Exception as e:
    print(f"⚠️ No challenger model found: {e}")
    challenger_version = None

In [ ]:
# Function to evaluate the metric values
def compute_metrics(y_true, y_pred, model_name):
    """Compute AUC, F1, and Accuracy scores"""
    auc = round(roc_auc_score(y_true, y_pred), 4)
    f1 = round(f1_score(y_true, y_pred), 4)
    accuracy = round(accuracy_score(y_true, y_pred), 4)
    
    print(f"\n📈 {model_name} Performance:")
    print(f"   🎯 AUC Score: {auc:.4f}")
    print(f"   🎯 F1 Score: {f1:.4f}")
    print(f"   🎯 Accuracy: {accuracy:.4f}")
    
    return {'auc': auc, 'f1': f1, 'accuracy': accuracy}


### Calculate the Champion and Challenger performance metric on hold out evaluation data.

In [ ]:
#Run Champion prediction function
y_pred_champion = model.version(live_version).run(X_eval, function_name="predict")
# 🏆 Champion Performance
champion_metrics = compute_metrics(y_true, y_pred_champion, "CHAMPION")

In [ ]:
#Run Challenger prediction function
y_pred_challenger = model.version(challenger_version).run(X_eval, function_name="predict")

# 🥊 Challenger Performance
challenger_metrics = compute_metrics(y_true, y_pred_challenger, "CHALLENGER")

# Swap models if Challenger score is higher

In [ ]:
print("🔄 EVALUATING MODEL PROMOTION")

if challenger_metrics['auc'] > champion_metrics['auc']:
    
    #Unset challenger alias
    model.version('CHALLENGER').unset_alias('CHALLENGER')

    #########  DEMOTE THE CURRENT MODEL ############
    model.version('CHAMPION').unset_alias('CHAMPION')
    
    ########## PROMOTE THE CURRENT CHALLENGER MODEL ############
    model.version(challenger_version).set_alias('CHAMPION')
    
    model.default = challenger_version #optional
    model.set_tag("live_version", challenger_version)
    print ("Challenger is promoted to Live")
    
    # Promotion metadata
    model.version(challenger_version).set_metric("promotion_date", str(datetime.now()))
    model.version(challenger_version).set_metric("promoted_from", live_version)
    model.version(challenger_version).set_metric("eval_auc", f"{challenger_metrics['auc']}")
        
    print(f"\n🎉 MODEL PROMOTION COMPLETED SUCCESSFULLY!")
    print(f"🏆 New Champion: {live_version}")
    
else:
    print(f"\n❌ PROMOTION CRITERIA NOT MET")
    print(f"📝 Reason: Challenger performance not better than champion")
    print(f"🏆 Current Champion Remains Active: {live_version}")
   